In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19793921
paper_name = 'kanki_klionsky_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/TableS2-2.xlsx', sheet_name='Sheet1', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4669 x 6


In [10]:
original_data.head()

,ORF name,Vacuolar GFP,Growth in YPL,OM45-GFP signal,Western blot,Gene
0,YMR158W-A,No,NaN,NaN,Fig. S2,Overlap with ATG16
1,YMR159C,No,NaN,NaN,Fig. S2,atg16
2,YPL166W,No,NaN,NaN,Fig. S2,atg29
3,YPL149W,No,Slow,Weak signal,Fig. S2,atg5
4,YPL120W,No,NaN,NaN,Fig. S2,atg6/vps30


In [11]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                      ORF name Vacuolar GFP  \
index_input                                                                   
4667                                                       NaN          NaN   
4668         x: cell did not grow in YPD medium or Om45-GFP...          NaN   

            Growth in YPL OM45-GFP signal Western blot Gene  \
index_input                                                   
4667                  NaN             NaN          NaN  NaN   
4668                  NaN             NaN          NaN  NaN   

                                                           orf  
index_input                                                     
4667                                                       NAN  
4668         XCELLDIDNOTGROWINYPDMEDIUMOROM45-GFPSIGNALCOUL...  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
data_dict = {'x': np.nan,'No':-1,'Yes':1}

In [17]:
original_data['data'] = original_data['Vacuolar GFP'].apply(lambda x: data_dict[x])

In [18]:
original_data.set_index('orf', inplace=True)

In [19]:
original_data = original_data[['data']].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(4592, 1)

In [25]:
original_data.loc[original_data['data']==0,'data'] = np.nan

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [160]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,160
data_type,value
orf,
YAL002W,1.0
YAL004W,1.0
YAL005C,1.0
YAL007C,1.0
YAL008W,-1.0


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,160
,data_type,value
gene_id,orf,
2,YAL002W,1.0
1863,YAL004W,1.0
4,YAL005C,1.0
5,YAL007C,1.0
6,YAL008W,-1.0


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        160          
data_type       value    valuez
gene_id orf                    
2       YAL002W   1.0  0.000000
1863    YAL004W   1.0  0.000000
4       YAL005C   1.0  0.000000
5       YAL007C   1.0  0.000000
6       YAL008W  -1.0 -3.364733

# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 19793921...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.53s/it]

Updating the data_modified_on field...
